# Extracting GeoTiff from Earth Engine using Landsat5
This notebook will help you learn about an GeoTiff and how to use this image type. In this notebook, you will learn the following:

* Accessing the google earth engine
* Extracting images from Google Earth Engine
* Displaying Satelite images on Maps
* Saving Google Earth images as GeoTiff to drive

### Folium
makes it easy to visualize data that’s been manipulated in Python on an interactive leaflet map.<br>
[For more info...](http://python-visualization.github.io/folium/)

**Python Installation:**

* Conda Package Manager: `conda install -c conda-forge folium`
* Python Package Installer: `pip install folium`

### EE 
It is the Google Earth Engine Python API. <br>
[For more info...](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)

**Python Installation:**

* [Conda Package Manager](https://developers.google.com/earth-engine/guides/python_install-conda#updating_the_api): `conda update -c conda-forge earthengine-api`
* [Python Package Installer](https://developers.google.com/earth-engine/guides/python_install#pip): `pip install earthengine-api --upgrade`

In [1]:
import folium
import ee

In [2]:
# Authenticate to the Earth Engine servers:
ee.Authenticate()


Successfully saved authorization token.


In [3]:
# Initialize the API:
ee.Initialize() 

### <font style='color:rgb(50,120,230)'> Function to mask clouds </font>

The following function is available at the the google earth engine landsat sections.
<hr style='border:none; height: 4px; background-color:#D3D3D3'/>

### Documentation
[`cloudMaskL457`](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_SR#description)

<hr style='border:none; height: 4px; background-color:#D3D3D3'/>

In [4]:
# Function to mask clouds based on the pixel_qa band of Landsat 4, 5 and 7 SR data.

def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    # Get the pixel QA band.
    mask = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask)


### <font style='color:rgb(50,120,230)'>Setting the parameters for getting data from Google Earth Engine.</font>

In [5]:
# The Region of interest, Cancún, Mexoco.
geometry = ee.Geometry.Polygon(
    [[  [-87.10988867671762,21.07449634501289],
        [-86.81669104488168,21.07449634501289],
        [-86.81669104488168,21.251229199817406],
        [-87.10988867671762,21.251229199817406]]], 
        None,
        False)

# Setting the parameters for Landsat5
visParams5 = {
  "bands": ['B3', 'B2', 'B1'],
  "min": 0,
  "max": 3000,
  "gamma": 1.4
}

dataset85 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1985-01-01', '1985-12-31').map(cloudMaskL457)
lansat85 = dataset85.median().clip(geometry)
dataset93 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1993-01-01', '1993-12-31').map(cloudMaskL457)
lansat93 = dataset93.median().clip(geometry)
dataset01 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2001-01-01', '2001-12-31').map(cloudMaskL457)
lansat01 = dataset01.median().clip(geometry)
dataset11 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2009-01-01', '2011-12-31').map(cloudMaskL457)
lansat11 = dataset11.median().clip(geometry)

# Using folium for getting an interactive map 

<hr style='border:none; height: 4px; background-color:#D3D3D3'/>

## Displaying Earth Engine image tiles to folium map
**`add_ee_layer()`** adds a method for displaying Earth Engine image tiles to folium map.

### Documentation
[`add_ee_layer()`](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)

## Drawing map
**`folium.map`** is the class for drawing maps.

### Documentation
[`folium.map`](http://python-visualization.github.io/folium/modules.html#module-folium.map)

<hr style='border:none; height: 4px; background-color:#D3D3D3'/>



In [6]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    '''Adds a method for displaying Earth Engine image tiles to folium map.'''
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

### <font style='color:rgb(50,120,230)'>Generating the Map with set center and zoom levels.</font>

In [7]:

map = folium.Map(location = (21.18, -86.95), zoom_start = 11)
# Adding the generated images onto the map.
map.add_ee_layer(lansat85, visParams5, '1985')
map.add_ee_layer(lansat93, visParams5, '1993')
map.add_ee_layer(lansat01, visParams5, '2001')
map.add_ee_layer(lansat11, visParams5, '2011')

# Add a layer control panel to the map.
map.add_child(folium.LayerControl())

# Display the map.
display(map)

### <font style='color:rgb(50,120,230)'>Creating the tasks to export the generated images from Landsat5</font>

In [8]:
task_config = {
  'folder': 'deforestation',
  'scale': 30,
  'region': geometry,
  'fileFormat': 'GeoTIFF',
  'maxPixels': 1e9,
  'formatOptions': {
    'cloudOptimized': True
  }
}
task1 = ee.batch.Export.image.toDrive(lansat85, description= '1985api', **task_config)
task2 = ee.batch.Export.image.toDrive(lansat93, description= '1993api', **task_config)
task3 = ee.batch.Export.image.toDrive(lansat01, description= '2001api', **task_config)
task4 = ee.batch.Export.image.toDrive(lansat11, description= '2011api', **task_config)

### <font style='color:rgb(50,120,230)'>Running the created tasks</font>

In [9]:
task1.start()
task2.start()
task3.start()
task4.start()

### Now that we have exported the images to our drive, we can download these images and perform further analysis on these GeoTIFF images.